# Setup

📝 *Import required libraries and files:*

In [1]:
import pandas as pd
import numpy as np

#    ---- Scrape Market Value ----
import requests, time, random, re
from bs4 import BeautifulSoup
import os, time, random, re
from pathlib import Path

In [2]:
path = "dfs/df_proc.parquet"
df_proc = pd.read_parquet(path)
pd.set_option('display.max_columns', None)
print(f"Shape of df: {df_proc.shape}")
df_proc.head(2)

Shape of df: (7583, 166)


,PlayerID,Player,League,Squad,elegible,Position,Pos,Age,Nation,Matches,Starts,Minutes,Goals,Expected Goals,Goals Minus Expected,Goals not Penalty,Penalty Goals,Penalty Efficacy,Shoots,Shoots on Target,Goals/Shoot,AvgShoot Distance,FreeKick Tacker,Assists,Crosses,Penalty Won,Offsides,Cmp Passes,Cmp Passes%,Prog Passes,Passing Distance,Pass Prog Distance,Short Cmp,Short Cmp%,Medium Cmp,Medium Cmp%,Long Cmp,Long Cmp%,Key Passes,Goal Creating Actions,Take-Ons Succ,Take-Ons Succ%,Fouls Suffered,Carries,Carries TotDist,Prog Carries,Carries PrgDist,Carries 1/3,Touches,Errors,Aerial Duels,Aerial Duels Won%,Tackles,Tackles Won,Blocks,Interceptions,Ball Recoveries,Clearances,Own Goals,Fouls Commited,Penalty Commited,Yellow Cards,Red Cards,Goals_90m,Expected Goals_90m,Goals Minus Expected_90m,Goals not Penalty_90m,Penalty Goals_90m,Shoots_90m,Shoots on Target_90m,Assists_90m,Crosses_90m,Penalty Won_90m,Offsides_90m,Cmp Passes_90m,Prog Passes_90m,Passing Distance_90m,Pass Prog Distance_90m,Short Cmp_90m,Medium Cmp_90m,Long Cmp_90m,Key Passes_90m,Goal Creating Actions_90m,Take-Ons Succ_90m,Fouls Suffered_90m,Carries_90m,Carries TotDist_90m,Prog Carries_90m,Carries PrgDist_90m,Carries 1/3_90m,Touches_90m,Errors_90m,Aerial Duels_90m,Tackles_90m,Tackles Won_90m,Blocks_90m,Interceptions_90m,Ball Recoveries_90m,Clearances_90m,Own Goals_90m,Fouls Commited_90m,Penalty Commited_90m,Yellow Cards_90m,Red Cards_90m,Goals_90m_scaled,Expected Goals_90m_scaled,Goals Minus Expected_90m_scaled,Goals not Penalty_90m_scaled,Penalty Goals_90m_scaled,Penalty Efficacy_scaled,Shoots_90m_scaled,Shoots on Target_90m_scaled,Goals/Shoot_scaled,AvgShoot Distance_scaled,Assists_90m_scaled,Crosses_90m_scaled,Penalty Won_90m_scaled,Offsides_90m_scaled,Cmp Passes_90m_scaled,Cmp Passes%_scaled,Prog Passes_90m_scaled,Passing Distance_90m_scaled,Pass Prog Distance_90m_scaled,Short Cmp_90m_scaled,Short Cmp%_scaled,Medium Cmp_90m_scaled,Medium Cmp%_scaled,Long Cmp_90m_scaled,Long Cmp%_scaled,Key Passes_90m_scaled,Goal Creating Actions_90m_scaled,Take-Ons Succ_90m_scaled,Take-Ons Succ%_scaled,Fouls Suffered_90m_scaled,Carries_90m_scaled,Carries TotDist_90m_scaled,Prog Carries_90m_scaled,Carries PrgDist_90m_scaled,Carries 1/3_90m_scaled,Touches_90m_scaled,Errors_90m_scaled,Aerial Duels_90m_scaled,Aerial Duels Won%_scaled,Tackles_90m_scaled,Tackles Won_90m_scaled,Blocks_90m_scaled,Interceptions_90m_scaled,Ball Recoveries_90m_scaled,Clearances_90m_scaled,Own Goals_90m_scaled,Fouls Commited_90m_scaled,Penalty Commited_90m_scaled,Yellow Cards_90m_scaled,Red Cards_90m_scaled,Goal Scoring,Goal Efficacy,Shooting,Passing Influence,Passing Accuracy,Goal Creation,Possession Influence,Progression,Dribling,Aerial Influence,Defensive Influence,Discipline and Consistency
0,1,Matías Abaldo,AR 1,Independiente,0,Forward,FW,21,URU,3,3,201,0,0.2,-0.2,0,0,NaN,2,1,0.00,16.7,0,0,3,0,3,45,70.3,8,597,231,29,80.6,13,72.2,2,50.0,1,4,1,12.5,3,50,226,3,82,0,96,0,6,50.0,2,1,3,1,7,0,0,0,0,0,0,0.000,0.090,-0.090,0.000,0.0,0.896,0.448,0.000,1.343,0.0,1.343,20.149,3.582,267.313,103.433,12.985,5.821,0.896,0.448,1.791,0.448,1.343,22.388,101.194,1.343,36.716,0.000,42.985,0.000,2.687,0.896,0.448,1.343,0.448,3.134,0.000,0.0,0.000,0.0,0.000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Lucas Abascia,AR 1,Cen. Córdoba–SdE,1,Defender,DF,30,ARG,14,11,1063,1,0.8,0.2,1,0,NaN,9,2,0.11,17.2,0,1,0,0,0,542,79.5,46,11663,4182,149,86.6,305,89.7,85,53.8,2,12,4,57.1,11,448,2377,7,1442,5,880,1,64,50.0,26,17,16,36,70,93,0,6,0,2,0,0.085,0.068,0.017,0.085,0.0,0.762,0.169,0.085,0.000,0.0,0.000,45.889,3.895,987.460,354.073,12.615,25.823,7.197,0.169,1.016,0.339,0.931,37.930,201.251,0.593,122.088,0.423,74.506,0.085,5.419,2.201,1.439,1.355,3.048,5.927,7.874,0.0,0.508,0.0,0.169,0.0,0.103,0.096,0.517,0.114,0.0,NaN,0.186,0.096,0.22,0

# 3.1. - Scrape Transfermarkt: Market Value and Position

In [3]:
def get_market_value_and_position(player_name: str):
    """
    Query transfermarkt.co.uk quick-search and return (market_value_str, position_str)
    for the FIRST player result. Example: ("€35.00m", "RW").
    """
    base_url = "https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-GB,en;q=0.9",
        "Referer": "https://www.transfermarkt.co.uk/",
    }
    params = {"query": player_name}

    def _norm(s: str) -> str:
        return re.sub(r"\s+", " ", s).strip().lower()

    try:
        r = requests.get(base_url, headers=headers, params=params, timeout=20)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        # pick the players results table by header names
        table = None
        for t in soup.select("table.items"):
            hdrs = [_norm(th.get_text(" ", strip=True)) for th in t.select("thead tr th")]
            if "position" in hdrs and ("market value" in hdrs or "marketvalue" in hdrs):
                table = t
                break
        if table is None:
            # fallback: first items table
            table = soup.select_one("table.items")
        if table is None:
            print(f"No results table found for: {player_name}")
            return None, None

        row = table.select_one("tbody > tr")
        if row is None:
            print(f"No player rows for: {player_name}")
            return None, None

        # Build header index mapping (robust to column order)
        hdrs = [_norm(th.get_text(" ", strip=True)) for th in table.select("thead tr th")]
        cells = row.select("td")

        def idx_for(name):
            try:
                return hdrs.index(name)
            except ValueError:
                return None

        pos_idx = idx_for("position")
        mv_idx  = idx_for("market value")

        # Primary extraction via header indices
        position = cells[pos_idx].get_text(" ", strip=True) if pos_idx is not None else None
        market_value = cells[mv_idx].get_text(" ", strip=True) if mv_idx is not None else None

        # Fallbacks if headers/indices fail
        if market_value in (None, ""):
            # Typical MV cell has classes 'rechts hauptlink' and contains € or £
            mv_td = row.select_one("td.rechts.hauptlink")
            market_value = mv_td.get_text(" ", strip=True) if mv_td else None
            if not market_value:
                for td in cells:
                    txt = td.get_text(" ", strip=True)
                    if "€" in txt or "£" in txt:
                        market_value = txt
                        break

        if position in (None, ""):
            # Look for a short positional code (GK/RB/LB/CB/DM/CM/AM/RW/LW/ST/CF…)
            for td in cells:
                txt = td.get_text(" ", strip=True)
                if re.fullmatch(r"[A-Z]{1,3}", txt):
                    position = txt
                    break

        # polite pause to be nice
        time.sleep(1 + random.uniform(0.2, 0.8))

        if not market_value and not position:
            print(f"Could not find fields for: {player_name}")
        return market_value, position

    except Exception as e:
        print(f"Error for {player_name}: {e}")
        return None, None

In [5]:
def _sleep_jitter(low=4, high=7):
    time.sleep(random.uniform(low, high))

def _safe_lookup(name, retries=3, base_wait=5):
    """
    Wraps your get_market_value_and_position with retries + backoff.
    Returns (mv, pos) or (None, None) on failure.
    """
    for attempt in range(retries):
        try:
            mv, pos = get_market_value_and_position(name)  # <-- YOUR function
            return mv, pos
        except Exception as e:
            wait = base_wait * (2 ** attempt) + random.uniform(0, 1.5)
            print(f"[WARN] {name}: {e} | retrying in {wait:.1f}s ({attempt+1}/{retries})")
            time.sleep(wait)
    return None, None

def scrape_tm_range(
    df: pd.DataFrame,
    *,
    start_id: int,
    end_id: int,
    id_col: str = "playerID",         
    name_col: str = "Player",
    eligible_col: str = "elegible",
    batch_size: int = 50,
    per_request_sleep=(4, 7),          
    sleep_between_batches: int = 60,   
    out_dir: str = "tm_exports",
    cumulative_file: str = "tm_cumulative.parquet",
    retries: int = 3,
    base_backoff: int = 5
) -> pd.DataFrame:
    """
    Scrape Position + Market Value from transfermarkt.co.uk for elegible players
    with IDs in [start_id, end_id]. Runs in batches of 50 by default and
    checkpoints to disk after each batch.

    Returns a DataFrame with columns:
        ['playerID','Player','Position','Market Value']
    """
    # --- prepare I/O
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)
    cum_path = out / cumulative_file

    # --- normalize inputs / select elegible rows within ID range
    if id_col not in df.columns and "PlayerID" in df.columns:
        id_col = "PlayerID"  # be forgiving if they use 'PlayerID'

    work = df[[id_col, name_col, eligible_col]].copy()
    work[id_col] = pd.to_numeric(work[id_col], errors="coerce")
    work = work.dropna(subset=[id_col])
    work = work[(work[eligible_col] == 1) & (work[id_col].between(start_id, end_id))]
    work = work.sort_values(id_col)

    if work.empty:
        print("No elegible players in the requested range.")
        return pd.DataFrame(columns=["playerID", "Player", "Position", "Market Value"])

    # --- load cumulative progress (resume support)
    if cum_path.exists():
        cumulative = pd.read_parquet(cum_path)
        done_ids = set(pd.to_numeric(cumulative.get(id_col, cumulative.get("playerID")), errors="coerce").dropna().astype(int))
        print(f"Resuming: {len(done_ids)} players already scraped in cumulative.")
    else:
        cumulative = pd.DataFrame(columns=[id_col, name_col, "Position", "Market Value"])
        done_ids = set()

    # --- compute pending IDs
    ids = work[id_col].astype(int).tolist()
    names = work.set_index(id_col)[name_col].to_dict()
    pending_ids = [pid for pid in ids if pid not in done_ids]
    total_pending = len(pending_ids)
    print(f"To process now: {total_pending} elegible players "
          f"({start_id}–{end_id}, batch={batch_size}).")

    # --- main loop over batches of 50
    low, high = per_request_sleep
    for b_start in range(0, total_pending, batch_size):
        b_ids = pending_ids[b_start:b_start + batch_size]
        if not b_ids:
            break

        bmin, bmax = min(b_ids), max(b_ids)
        batch_fname = out / f"tm_{bmin:05d}_{bmax:05d}.parquet"

        # If the exact batch file already exists, merge it into cumulative and skip scraping
        if batch_fname.exists():
            print(f"Batch {bmin}-{bmax} already exists → merging into cumulative and skipping scrape.")
            try:
                existing = pd.read_parquet(batch_fname)
                cumulative = (
                    pd.concat([cumulative, existing], ignore_index=True)
                      .drop_duplicates(subset=[id_col], keep="last")
                )
                cumulative.to_parquet(cum_path, index=False)
            except Exception as e:
                print(f"[WARN] Could not read existing batch file {batch_fname.name}: {e}")
            time.sleep(sleep_between_batches)
            continue

        print(f"Batch {bmin}-{bmax} | {len(b_ids)} players")
        records = []

        for pid in b_ids:
            name = names.get(pid)
            if not name:
                continue

            # Skip if already have a non-null MV recorded for this ID
            if pid in done_ids:
                continue

            mv, pos = _safe_lookup(name, retries=retries, base_wait=base_backoff)
            records.append({id_col: pid, name_col: name, "Position": pos, "Market Value": mv})

            _sleep_jitter(low, high)  # polite pacing

        # Save batch immediately
        batch_df = pd.DataFrame(records)
        if not batch_df.empty:
            batch_df.to_parquet(batch_fname, index=False)

            # Update cumulative and save
            cumulative = (
                pd.concat([cumulative, batch_df], ignore_index=True)
                  .drop_duplicates(subset=[id_col], keep="last")
            )
            cumulative.to_parquet(cum_path, index=False)

        print(f"Saved {batch_fname.name} and updated {cum_path.name}.")
        print(f"Sleeping {sleep_between_batches}s before next batch…")
        time.sleep(sleep_between_batches)

    print("All requested batches complete.")

    # --- Final return: ensure required column names/order and rename id to 'playerID'
    out_df = cumulative[[id_col, name_col, "Position", "Market Value"]].copy()
    out_df = out_df.rename(columns={id_col: "playerID"})
    out_df = out_df.sort_values("playerID").reset_index(drop=True)

    # Return only rows in the requested range & elegible
    out_df = out_df[out_df["playerID"].between(start_id, end_id)]
    out_df = out_df.merge(
        work[[id_col]].rename(columns={id_col: "playerID"}), on="playerID", how="inner"
    )

    return out_df[["playerID", "Player", "Position", "Market Value"]]

Scraping in 4 batches.

In [6]:
result_batch1 = scrape_tm_range(
    df_scaled,
    start_id=1,
    end_id=2100,
    id_col="PlayerID",        
    name_col="Player",
    eligible_col="elegible",
    batch_size=50,
    per_request_sleep=(4, 7),
    sleep_between_batches=60,
)

Resuming: 100 players already scraped in cumulative.
To process now: 1185 elegible players (1–2100, batch=50).
Batch 166-246 | 50 players
Saved tm_00166_00246.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 248-328 | 50 players
No results table found for: Juan Franco Arrellaga
Saved tm_00248_00328.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 329-409 | 50 players
No results table found for: Pablo Galdames Millán
No results table found for: Laurtaro Giaccone
Could not find fields for: Jonatan Goitía
No results table found for: Carlos González Espínola
Saved tm_00329_00409.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 410-498 | 50 players
Saved tm_00410_00498.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 499-576 | 50 players
Could not find fields for: Juan Ludueña
Saved tm_00499_00576.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next 

In [7]:
result_batch2 = scrape_tm_range(
    df_scaled,
    start_id=2101,
    end_id=4200,
    id_col="PlayerID",        
    name_col="Player",
    eligible_col="elegible",
    batch_size=50,
    per_request_sleep=(4, 7),
    sleep_between_batches=60,
)

Resuming: 1285 players already scraped in cumulative.
To process now: 1460 elegible players (2101–4200, batch=50).
Batch 2102-2188 | 50 players
No results table found for: Guilherme Tinga
Could not find fields for: Félix Torres Caicedo
Saved tm_02102_02188.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 2189-2256 | 50 players
No results table found for: Victor Bernth Kristiansen
Saved tm_02189_02256.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 2257-2329 | 50 players
Saved tm_02257_02329.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 2331-2403 | 50 players
No results table found for: Hwang Hee-chan
No results table found for: Son Heung-min
Saved tm_02331_02403.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 2404-2469 | 50 players
Saved tm_02404_02469.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 2470-2541 | 50 players


In [8]:
result_batch3 = scrape_tm_range(
    df_scaled,
    start_id=4201,
    end_id=6300<,
    id_col="PlayerID",        
    name_col="Player",
    eligible_col="elegible",
    batch_size=50,
    per_request_sleep=(4, 7),
    sleep_between_batches=60,
)

Resuming: 2745 players already scraped in cumulative.
To process now: 1458 elegible players (4201–6300, batch=50).
Batch 4201-4265 | 50 players
No results table found for: Hong Hyunseok
No results table found for: Lee Jae-sung
Saved tm_04201_04265.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 4266-4334 | 50 players
No results table found for: Giorgos Masouras
No results table found for: Jean Mattéo Bahoya
No results table found for: Kim Min-jae
Saved tm_04266_04334.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 4335-4410 | 50 players
No results table found for: Joakim Mæhle
No results table found for: Nathan N'Goumou
No results table found for: Stanley N'Soki
Saved tm_04335_04410.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 4411-4475 | 50 players
Saved tm_04411_04475.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 4476-4547 | 50 players
No results ta

In [9]:
result_batch4 = scrape_tm_range(
    df_scaled,
    start_id=6300,
    end_id=8367,
    id_col="PlayerID",        
    name_col="Player",
    eligible_col="elegible",
    batch_size=50,
    per_request_sleep=(4, 7),
    sleep_between_batches=60,
)

Resuming: 4203 players already scraped in cumulative.
To process now: 1410 elegible players (6300–8367, batch=50).
Batch 6301-6371 | 50 players
Saved tm_06301_06371.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 6372-6453 | 50 players
No results table found for: Uroš Đurđević
Error for Amar Ahmed: 503 Server Error: Service Unavailable for url: https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=Amar+Ahmed
Error for Makan Aiko: 503 Server Error: Service Unavailable for url: https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=Makan+Aiko
Error for Chuba Akpom: 503 Server Error: Service Unavailable for url: https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query=Chuba+Akpom
Saved tm_06372_06453.parquet and updated tm_cumulative.parquet.
Sleeping 60s before next batch…
Batch 6454-6529 | 50 players
Error for Paweł Bochniewicz: 503 Server Error: Service Unavailable for url: https://www.transfermarkt.c

# 3.2. - Consolidate Data

In [3]:
df_tm = pd.read_parquet("tm_exports/tm_cumulative.parquet")

In [4]:
print(f"transfermarkt df shape: {df_tm.shape}")
df_tm.head(5)

transfermarkt df shape: (5613, 4)


,PlayerID,Player,Position,Market Value
0,2,Lucas Abascia,CB,€800k
1,5,Ignacio Abraham,LB,€300k
2,6,Gonzalo Abrego,CM,€2.00m
3,7,Jonás Acevedo,LW,€1.00m
4,9,Guillermo Acosta,CM,€200k


In [5]:
df_final = df_proc.merge(
    df_tm[['PlayerID', 'Position', 'Market Value']],
    on='PlayerID', how='left'
)

In [6]:
# Drop 'Pos' column (redundant now that we have the detail from TM) + Rename Position columns
df_final = (
    df_final
      .drop(columns=["Pos"], errors="ignore")      
      .rename(columns={"Position_x": "Position_FBRef","Position_y": "Position_TM"})            # replace with your rename mapping
)

In [7]:
# 1. Adjust 'unclear' positions
# --- RM -> RB if FBRef==Defender else RW
m = df_final["Position_TM"].eq("RM")
df_final.loc[m & df_final["Position_FBRef"].eq("Defender"), "Position_TM"] = "RB"
df_final.loc[m & ~df_final["Position_FBRef"].eq("Defender"), "Position_TM"] = "RW"

# --- LM -> LB if FBRef==Defender else LW
m = df_final["Position_TM"].eq("LM")
df_final.loc[m & df_final["Position_FBRef"].eq("Defender"), "Position_TM"] = "LB"
df_final.loc[m & ~df_final["Position_FBRef"].eq("Defender"), "Position_TM"] = "LW"

# --- SS -> CF if FBRef==Forward else AM
m = df_final["Position_TM"].eq("SS")
df_final.loc[m & df_final["Position_FBRef"].eq("Forward"), "Position_TM"] = "CF"
df_final.loc[m & ~df_final["Position_FBRef"].eq("Forward"), "Position_TM"] = "AM"

# --- Invalid Positions
valid_positions = ["CB","RB","LB","DM","CM","AM","RW","LW","CF"]
df_final.loc[~df_final["Position_TM"].isin(valid_positions), "Position_TM"] = "Undefined"

df_final["Position"] = df_final["Position_TM"].fillna(df_final["Position_TM"])

# 3. Fallback to FBRef if missing/unmapped position
mask = df_final["Position"].isna() | (df_final["Position"] == "") | (df_final["Position"] == "Undefined")
df_final.loc[mask, "Position"] = df_final.loc[mask, "Position_FBRef"]

In [8]:
df_final = df_final.drop(columns=["Position_FBRef","Position_TM"], errors="ignore")    

In [9]:
# Adjust column order
cols = [c for c in df_final.columns if c not in ["Position", "Market Value"]]
cols.insert(5, "Position")     
cols.insert(8, "Market Value") 
df_final = df_final[cols]

In [10]:
# Convert Market Value to numeric
def convert_market_value(val):
    if pd.isna(val):
        return None
    val = str(val).replace("€", "").strip().lower()
    if val.endswith("m"):
        return float(val.replace("m", ""))  # already in millions
    elif val.endswith("k"):
        return float(val.replace("k", "")) / 1000  # convert thousands to millions
    else:
        return None 

df_final["Market Value"] = df_final["Market Value"].apply(convert_market_value).round(1)
df_final = df_final.rename(columns={"Market Value": "Market Value (M€)"})

In [11]:
df_final.head(10)

,PlayerID,Player,League,Squad,elegible,Position,Age,Nation,Market Value (M€),Matches,Starts,Minutes,Goals,Expected Goals,Goals Minus Expected,Goals not Penalty,Penalty Goals,Penalty Efficacy,Shoots,Shoots on Target,Goals/Shoot,AvgShoot Distance,FreeKick Tacker,Assists,Crosses,Penalty Won,Offsides,Cmp Passes,Cmp Passes%,Prog Passes,Passing Distance,Pass Prog Distance,Short Cmp,Short Cmp%,Medium Cmp,Medium Cmp%,Long Cmp,Long Cmp%,Key Passes,Goal Creating Actions,Take-Ons Succ,Take-Ons Succ%,Fouls Suffered,Carries,Carries TotDist,Prog Carries,Carries PrgDist,Carries 1/3,Touches,Errors,Aerial Duels,Aerial Duels Won%,Tackles,Tackles Won,Blocks,Interceptions,Ball Recoveries,Clearances,Own Goals,Fouls Commited,Penalty Commited,Yellow Cards,Red Cards,Goals_90m,Expected Goals_90m,Goals Minus Expected_90m,Goals not Penalty_90m,Penalty Goals_90m,Shoots_90m,Shoots on Target_90m,Assists_90m,Crosses_90m,Penalty Won_90m,Offsides_90m,Cmp Passes_90m,Prog Passes_90m,Passing Distance_90m,Pass Prog Distance_90m,Short Cmp_90m,Medium Cmp_90m,Long Cmp_90m,Key Passes_90m,Goal Creating Actions_90m,Take-Ons Succ_90m,Fouls Suffered_90m,Carries_90m,Carries TotDist_90m,Prog Carries_90m,Carries PrgDist_90m,Carries 1/3_90m,Touches_90m,Errors_90m,Aerial Duels_90m,Tackles_90m,Tackles Won_90m,Blocks_90m,Interceptions_90m,Ball Recoveries_90m,Clearances_90m,Own Goals_90m,Fouls Commited_90m,Penalty Commited_90m,Yellow Cards_90m,Red Cards_90m,Goals_90m_scaled,Expected Goals_90m_scaled,Goals Minus Expected_90m_scaled,Goals not Penalty_90m_scaled,Penalty Goals_90m_scaled,Penalty Efficacy_scaled,Shoots_90m_scaled,Shoots on Target_90m_scaled,Goals/Shoot_scaled,AvgShoot Distance_scaled,Assists_90m_scaled,Crosses_90m_scaled,Penalty Won_90m_scaled,Offsides_90m_scaled,Cmp Passes_90m_scaled,Cmp Passes%_scaled,Prog Passes_90m_scaled,Passing Distance_90m_scaled,Pass Prog Distance_90m_scaled,Short Cmp_90m_scaled,Short Cmp%_scaled,Medium Cmp_90m_scaled,Medium Cmp%_scaled,Long Cmp_90m_scaled,Long Cmp%_scaled,Key Passes_90m_scaled,Goal Creating Actions_90m_scaled,Take-Ons Succ_90m_scaled,Take-Ons Succ%_scaled,Fouls Suffered_90m_scaled,Carries_90m_scaled,Carries TotDist_90m_scaled,Prog Carries_90m_scaled,Carries PrgDist_90m_scaled,Carries 1/3_90m_scaled,Touches_90m_scaled,Errors_90m_scaled,Aerial Duels_90m_scaled,Aerial Duels Won%_scaled,Tackles_90m_scaled,Tackles Won_90m_scaled,Blocks_90m_scaled,Interceptions_90m_scaled,Ball Recoveries_90m_scaled,Clearances_90m_scaled,Own Goals_90m_scaled,Fouls Commited_90m_scaled,Penalty Commited_90m_scaled,Yellow Cards_90m_scaled,Red Cards_90m_scaled,Goal Scoring,Goal Efficacy,Shooting,Passing Influence,Passing Accuracy,Goal Creation,Possession Influence,Progression,Dribling,Aerial Influence,Defensive Influence,Discipline and Consistency
0,1,Matías Abaldo,AR 1,Independiente,0,Forward,21,URU,NaN,3,3,201,0,0.2,-0.2,0,0,NaN,2,1,0.00,16.7,0,0,3,0,3,45,70.3,8,597,231,29,80.6,13,72.2,2,50.0,1,4,1,12.5,3,50,226,3,82,0,96,0,6,50.0,2,1,3,1,7,0,0,0,0,0,0,0.000,0.090,-0.090,0.000,0.0,0.896,0.448,0.000,1.343,0.0,1.343,20.149,3.582,267.313,103.433,12.985,5.821,0.896,0.448,1.791,0.448,1.343,22.388,101.194,1.343,36.716,0.000,42.985,0.000,2.687,0.896,0.448,1.343,0.448,3.134,0.000,0.000,0.000,0.0,0.000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Lucas Abascia,AR 1,Cen. Córdoba–SdE,1,CB,30,ARG,0.8,14,11,1063,1,0.8,0.2,1,0,NaN,9,2,0.11,17.2,0,1,0,0,0,542,79.5,46,11663,4182,149,86.6,305,89.7,85,53.8,2,12,4,57.1,11,448,2377,7,1442,5,880,1,64,50.0,26,17,16,36,70,93,0,6,0,2,0,0.085,0.068,0.017,0.085,0.0,0.762,0.169,0.085,0.000,0.0,0.000,45.889,3.895,987.460,354.073,12.615,25.823,7.197,0.169,1.016,0.339,0.931,37.930,201.251,0.593,122.088,0.423,74.506,0.085,5.419,2.201,1.439,1.355,3.048,5.927,7.874,0.000,0.508,0.0,0.169,0.0,0.103,0.096,0.517,0.114,0.0,NaN,0.18

In [12]:
df_elegible = df_final[df_final['elegible']==1]
df_elegible['Position'].value_counts().sum

<bound method Series.sum of Position
CB            925
CM            594
CF            584
RB            425
DM            418
LB            384
AM            346
RW            343
LW            302
Defender       57
Midfielder     46
Forward        43
Name: count, dtype: int64>

# Save

**Save file** as `df_proc`:

In [13]:
df_final.to_parquet("dfs/df_final.parquet", index=False)
df_final.to_parquet("../assets/df_final.parquet", index=False)